In [2]:
import pyspark 
from pyspark.sql import SparkSession,Row,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *


spark=SparkSession.builder.master("local[1]")\
        .appName("movie_local")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.4.4/jars/mysql-connector-j-9.1.0.jar" ) \
        .config("spark.sql.warehouse.dir", "/home/jovyan/database") \
        .enableHiveSupport()\
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark


PySpark Version :3.5.0
PySpark Version :3.5.0


In [3]:
spark.sql("CREATE DATABASE IF NOT EXISTS db150")


DataFrame[]